In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout
from tensorflow.keras.applications import MobileNetV2, densenet , resnet50, VGG16


In [ ]:
# Constants
DATA_DIR = '../input/dog-breed-identification/'
TRAIN_DIR = DATA_DIR + 'train/'
TEST_DIR = DATA_DIR + 'test/'
BATCH_SIZE = 32
INPUT_SIZE = 224
NUM_CLASSES = 120
SEED = 42

In [ ]:
! ls $DATA_DIR

In [ ]:
import pandas as pd
train = pd.read_csv(DATA_DIR + 'labels.csv')
train.head()

In [ ]:
def append_ext(fn):
    return fn+".jpg"
train["id"]=train["id"].apply(append_ext)

In [ ]:
breeds_20 = train.breed.value_counts().head(NUM_CLASSES).index
train = train[train.breed.isin(breeds_20)]

In [ ]:
breeds_20

In [ ]:
train

In [ ]:
submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')
submission.head()

In [ ]:
breed_labels = list(breeds_20.values)
# print(len(breed_labels))
train.head()

In [ ]:
def create_model(lr=0.0001, dropout=None):
    
    base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(INPUT_SIZE, INPUT_SIZE, 3))
    for layer in base.layers:
        layer.trainable = False
    headmodel = base.output
    headmodel = Flatten()
    headmodel = tf.keras.layers.Dense(NUM_CLASSES, activation= 'softmax')(headmodel)
    model = tf.keras.Model(inputs=base.input, outputs= headmodel)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr) ,
                  loss = tf.keras.losses.categorical_crossentropy, metrics=['accuracy'],  )

    return model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
train_set, val_set = train_test_split(train, test_size=0.20, stratify=train['breed'], random_state=SEED)

In [ ]:
train_set.head()

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255.,
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe=train_set,
                                                directory=TRAIN_DIR,
                                                x_col='id',
                                                y_col='breed',
                                                class_mode='categorical',
                                                classes=breed_labels,
                                                has_ext=False,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                                seed=SEED,
                                                target_size=(INPUT_SIZE, INPUT_SIZE)
                                               )

In [ ]:
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255.
)

In [ ]:
val_generator = val_datagen.flow_from_dataframe(dataframe=val_set,
                                                directory=TRAIN_DIR,
                                                x_col='id',
                                                y_col='breed',
                                                class_mode='categorical',
                                                classes=breed_labels,
                                                has_ext=False,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                                seed=SEED,
                                                target_size=(INPUT_SIZE, INPUT_SIZE)
                                               )

In [ ]:
model = create_model()

In [ ]:
model.summary()

In [ ]:
EPOCHS = 50
model.fit(train_generator, epochs=EPOCHS, validation_data=val_generator,validation_freq=1)